# Alert Agent

Decides whether to alert based on weather data

This Agent:

- Has a **tool** to **send an alert** (dummy print) if temperature is higher than 25 degrees.
- Has no LLM
- Implements the `handle_task` function, which receives tasks from other agents.
- Implements the `send_task_to` function, which sends tasks to other agents. In this example it sends a task to the Weather Agent.

Uses an **HTTPRegistry** to register with the Registry and an **HTTPAgentTransport** for A2A communication.

**Note**: While in the Weather Agent we passed the registry url directly as `str` to `registry` argument of the agent, here we'll create a registry object and pass it directly to the Agent. The agent will actually export the RegistryClient form the Registry object.

In [ ]:
from protolink.agents import Agent
from protolink.models import Message, Task
from protolink.transport import HTTPAgentTransport, HTTPRegistryTransport

# Alert Agent URL
URL = "http://localhost:8020"

# This could be an env variable
REGISTRY_URL = "http://localhost:9000"

# HTTP A2A Trasport
transport = HTTPAgentTransport(url=URL)

# HTTP Agent-to-Registry Transport
registry = HTTPRegistryTransport(url=REGISTRY_URL)

### Define the Agent Class

The agent inherits from the `Agent` class and implements the `handle_task` and `send_task_to` methods, which are **mandatory** to implement if the Agent is expected to handle tasks from and send tasks to other Agents. 

In [ ]:
class AlertAgent(Agent):
    # Override handle_task
    async def handle_task(self, task: Task):
        data = task.payload
        if data["temperature"] > 25:
            await self.call_tool("alert_tool", message=f"Hot weather in {data['city']}! {data['temperature']}°C")
        return task

    async def send_task_to(self, agent_url: str, task: Task) -> Task:
        """Send a task to another agent.

        Args:
            agent_url: URL of the target agent
            task: Task to send

        Returns:
            Task with response from target agent

        Raises:
            RuntimeError: If no transport is configured
        """
        if not self._client:
            raise RuntimeError("No transport client configured. Call set_transport() first.")
        return await self._client.send_task(agent_url, task)

### Create Agent object

Let's pass the agent card to the Agent directly as a **dict** this time, without using the `AgentCard` class.

In [ ]:
# Define Agent Card using the a dict (Agent will cast it to AgentCard)
card = {"url": URL, "name": "AlertAgent", "description": "Sends alerts based on data"}

agent = AlertAgent(card=card, transport=transport, registry=registry)

### Add Agent native Tool

In [ ]:
# Add Native tool using the decorator
@agent.tool(name="alert_tool", description="Send an alert")
async def send_alert(message: str):
    print(f"ALERT: {message}")
    return {"status": "sent", "message": message}

### Send Task to Agent

The `send_task_to` method is used to send a task to another agent. In an A2A automated system the Agent decides on its own whether to send a task to another agent or not, and to which agent. Now we'll use it manually. This function uses the **AgentClient** to send a task to another agent. 

In [ ]:
task = Task.create(Message.user("Whats the temp in Geneva"))

WEATHER_AGENT_URL = "http://localhost:8010"

try:
    res = await agent.send_task_to(WEATHER_AGENT_URL, task)
except ConnectionError as e:
    print(f"Cannot reach target agent: {e}")

print(f"Weather Agent Response:\n{res.to_dict()}")